In [ ]:
# Implement code for configuring GPU and parallel processing for images
# Implement code for creating testing, validation and training dataset using the
# flow_from_dataframe API from tensorflow

In [ ]:
# !pip install tqdm
# !pip install tensorflow

In [5]:
import pandas as pd
import os
import shutil
from tqdm.notebook import tqdm as tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [20]:
df = pd.read_csv('D:/temp/train.csv',dtype=str)
print("Original shape: {}".format(df.shape))
df['filename'] = ["".join([str(df['patient_id'][i]),'_',str(df['image_id'][i]),'.png']) \
                  for i in range(len(df))]
df = df.loc[:,['filename','cancer']]
df.head()

Original shape: (54706, 14)


,filename,cancer
0,10006_462822612.png,0
1,10006_1459541791.png,0
2,10006_1864590858.png,0
3,10006_1874946579.png,0
4,10011_220375232.png,0


In [21]:
df['cancer'].unique()

array(['0', '1'], dtype=object)

In [3]:
def load_img(path):
    
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels = 1)
    img = tf.image.resize(img, [IMAGE_SIZE, IMAGE_SIZE])
    img = tf.cast(img, dtype = tf.float32)
    img = img/255.0
    return img

In [22]:
path = 'D:/temp/train_new/'
IMAGE_SIZE=256
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
datagen.flow_from_dataframe(df,
    directory=path,
    x_col='filename',
    y_col='cancer',
    weight_col=None,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb',
    class_mode='binary',
    batch_size=10,
    shuffle=True,
    save_format='png',
    subset='training',
    interpolation='nearest',
    validate_filenames=True,
)
# x_train,x_test,y_train,y_test = train_test_split(df.filename,df.cancer,stratify=df.cancer,test=0.20)
# model.fit(datagen.flow(x_train, y_train, batch_size=32,
#          subset='training'),
#          validation_data=datagen.flow(x_train, y_train,
#          batch_size=8, subset='validation'),
#          steps_per_epoch=len(x_train) / 32, epochs=epochs)

Found 54706 validated image filenames belonging to 2 classes.
